In [ ]:
from torchvision import transforms
import matplotlib.pyplot as plt
import torchvision.utils as vutils
from torchvision.utils import save_image
import pandas as pd
from PIL import Image
import numpy as np
import torch
import json
import cv2
import os

In [ ]:
root = "/datasets/cpg0017-rohban-pathways/2013_10_11_SIGMA2_Pilot/"

Read metadata file containing cell painting channel addresses for all replicates/experiments

In [ ]:
# loop over folders in root
df = pd.DataFrame()
for folder in os.listdir(root):
    print(folder)
    df_tmp = pd.read_csv(root + folder + "/load_data_with_illum.csv")
    if df.empty:
        df = df_tmp
        print(df.columns)
    else:
        df = pd.concat([df, df_tmp])

In [ ]:
# get unique values of Metadata_Well column with counts of each value
well_counts = df["Metadata_Well"].value_counts()
# print min, max, and mean of well counts
print('Min: ', well_counts.min())
print('Max: ', well_counts.max())
print('Mean: ', well_counts.mean())

Read metadata file which contains well_position, gene_name, pert_name, broad_sample, cell_line, ASSAY_WELL_ROLE, and GeneID information

In [ ]:
metadata_df = pd.read_csv("/datasets/cpg0017-rohban-pathways/platemaps/2013_10_11_SIGMA2_Pilot/platemap/TAORF_REFERENCE_SET_2.txt", sep="\t")
# rename well_position column name with Metadata_Well
metadata_df = metadata_df.rename(columns={"well_position": "Metadata_Well"})
metadata_df

Merge the dataframe containing channel addresses with the dataframe containing well info and gene names

In [ ]:
df_merged = pd.merge(df, metadata_df, on="Metadata_Well")
df_merged

In [ ]:
def create_folder(root, experiment):
    """Create folders for resized and cropped images

    Args:
        root (str): root directory
        experiment (str): experiment name
    """
    folder_list = ['generated_imgs', 'ground_truth', 'train_imgs']
    # create folder root+experiment+'_resized' and root+experiment+'_cropped' if they don't exist
    experiment_path = [root+experiment+'_resized',
                       root+experiment+'_cropped']
    for path in experiment_path:
        if not os.path.exists(path):
            os.makedirs(path)
            for folder in folder_list:
                if not os.path.exists(path + '/' + folder):
                    os.makedirs(path + '/' + folder)
    return

In [ ]:
def get_illuminated_corrected_channel(row, channel, img_root, illum_root):
    """ Responsible for loading the image and illumination correction file and correcting the pixel values.

    Args:
        row (pd.DataFrame): pd dataframe row
        channel (str): name of channel
        img_root (str): address of the image root
        illum_root (str): address of the illumination correction root

    Returns:
        np.ndarray: illuminated corrected image
    """
    channel_file = row["FileName_Orig"+channel]
    channel_path = row["PathName_Orig"+channel]
    batch = channel_path.split("/")[-1]
    channel_address = img_root + batch + "/" + channel_file
    img = Image.open(channel_address)
    img = np.asarray(img)
    img = cv2.normalize(
        img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            
    # load illumination correction file
    illum_file = row["FileName_Illum"+channel]
    illum_address = illum_root + batch + "/" + illum_file   
    # read .npy file 
    illum = np.load(illum_address)

    # apply illumination correction
    img = img / illum
    assert img.min() >= 0 and img.max() <= 255
    return img

In [ ]:
def save_resized_img(img, experiment, row):
    """ Responsible for resizing the image and saving it in the appropriate folder.

    Args:
        img (torch.tensor): illumination corrected image
        experiment (str): name of the experiment
    """
    perturbation = row['gene_name'].lower()
    # check if perturbation folder exists in the experiment dictionary and create it if it doesn't
    if not os.path.exists(experiment + '_resized/ground_truth/' + perturbation):
        os.makedirs(experiment + '_resized/ground_truth/' + perturbation)

    if not os.path.exists(experiment + '_resized/train_imgs/train'):
        os.makedirs(experiment + '_resized/train_imgs/train')

    transform = transforms.Resize((512, 512))
    img = img.permute(2, 0, 1)
    img_resized = transform(img).numpy().transpose(1, 2, 0)
    
    sample_name = '_'.join(row['FileName_OrigAGP'].split('_')[:3])+'_'+row['PathName_OrigAGP'].split('/')[-1]

    files = os.listdir(experiment + '_resized/train_imgs/train/')
    if sample_name+'.png' in files:
        print('Image already exists')
        print(sample_name+'.png')
        return
    
    if img_resized.max() < 60:
        print('Image is too dark')
        return
    cv2.imwrite(
        experiment+'_resized/ground_truth/'+perturbation+'/'+sample_name+'.png', img_resized)
    cv2.imwrite(
        experiment+'_resized/train_imgs/train/'+sample_name+'.png', img_resized)
    
    line = {
        "file_name": sample_name+'.png',
        "additional_feature": perturbation
    }
    
    if not os.path.exists(experiment + '_resized/train_imgs/train/metadata.jsonl'):
        with open(experiment + '_resized/train_imgs/train/metadata.jsonl', 'w') as f:
            f.write(json.dumps(line) + '\n')
    else:
        with open(experiment + '_resized/train_imgs/train/metadata.jsonl', 'a') as f:
            f.write(json.dumps(line) + '\n')
    # print()
    return

In [ ]:
def save_cropped_img(img, experiment, row):
    """ Responsible for cropping the image and saving it in the appropriate folder.
    
    Args:
        img (torch.tensor): illumination corrected image
        experiment (str): name of the experiment
        row (pd.DataFrame): pd dataframe row
    """
    perturbation = row['gene_name'].lower()
    # check if perturbation folder exists in the experiment dictionary and create it if it doesn't
    if not os.path.exists(experiment + '_cropped/ground_truth/' + perturbation):
        os.makedirs(experiment + '_cropped/ground_truth/' + perturbation)

    if not os.path.exists(experiment + '_cropped/train_imgs/train'):
        os.makedirs(experiment + '_cropped/train_imgs/train')
        
    sample_name = '_'.join(row['FileName_OrigAGP'].split('_')[:3])+'_'+row['PathName_OrigAGP'].split('/')[-1]
    files = os.listdir(experiment+'_cropped/train_imgs/train/')
    if sample_name+'.png' in files:
        print('Image already exists')
        print(sample_name+'.png')
        return

    # crop the image
    transform = transforms.FiveCrop((512, 512))
    img = img.permute(2, 0, 1)
    cropped_imgs = transform(img)
    for idx in range(len(cropped_imgs)):
        img = cropped_imgs[idx].numpy().transpose(1, 2, 0)

        if img.max() < 60:
            continue
        cv2.imwrite(
            experiment+'_cropped/ground_truth/'+perturbation+'/'+sample_name+'_'+str(idx)+'.png', img)
        cv2.imwrite(
            experiment+'_cropped/train_imgs/train/'+sample_name+'_'+str(idx)+'.png', img)
        
        line = {
            "file_name": sample_name+'_'+str(idx)+'.png',
            "additional_feature": perturbation
        }
        
        if not os.path.exists(experiment+'_cropped/train_imgs/train/metadata.jsonl'):
            with open(experiment+'_cropped/train_imgs/train/metadata.jsonl', 'w') as f:
                f.write(json.dumps(line) + '\n')
        else:
            with open(experiment+'_cropped/train_imgs/train/metadata.jsonl', 'a') as f:
                f.write(json.dumps(line) + '\n')

    return
    

In [ ]:
def correct_illumination(df, channel_list, experiment):
    """ Responsible for correcting the illumination of the images and saving them in the appropriate folders.
    
    Args:
        df (pd.DataFrame): pd dataframe
        channel_list (list): list of channel names
        experiment (str): name of the experiment
    """
    
    root = '/datasets/cpg0017-rohban-pathways/'
    create_folder(root, experiment)
    img_root = root + 'images/'
    illum_root = root + 'illum/'

    # loop over rows in df
    for index, row in df.iterrows():
        # loop over channels in channel_list
        ch1 = get_illuminated_corrected_channel(
            row, channel_list[0], img_root, illum_root)
        ch2 = get_illuminated_corrected_channel(
            row, channel_list[1], img_root, illum_root)
        ch3 = get_illuminated_corrected_channel(
            row, channel_list[2], img_root, illum_root)

        merged_image = np.dstack((ch1, ch2, ch3))
        img = torch.from_numpy(merged_image)

        save_resized_img(img, root+experiment, row)
        save_cropped_img(img, root+experiment, row)
    return

In [ ]:
def create_experiment_directories(experiment, gene_list, channel_list):
    """ Responsible for creating directories for the experiment and calling the correct_illumination function.
    
    Args:
        experiment (str): name of the experiment
        gene_list (list): list of gene names
        channel_list (list): list of channel names
    """
    # lowercase the gene names
    df_merged["gene_name"] = df_merged["gene_name"].str.lower()
    gene_list = [gene.lower() for gene in gene_list]

    # extract rows with gene_list in the gene_name column
    experiment_df = df_merged[df_merged["gene_name"].isin(gene_list)]
    experiment = experiment+'_'+'_'.join(channel_list)
    print('Experiment name:', experiment)
    print('Number of images for each gene in '+experiment+': ')
    print(experiment_df["Metadata_Well"].value_counts())
    print(experiment_df["gene_name"].value_counts())
    print(experiment_df.shape)
    correct_illumination(experiment_df, channel_list, experiment)
    return

In [ ]:
import os
import random
from PIL import Image

def create_collage(experiment, channel_list, preprocessing):
    """ Create a 10x10 collage of 100 randomly selected images from the training set.
    
    Args:
        experiment (str): name of the experiment
        channel_list (list): list of channels
        preprocessing (str): preprocessing method
    """
    directory = '/datasets/cpg0017-rohban-pathways/'+ \
        experiment+'_'+'_'.join(channel_list)+'_'+preprocessing+ \
        '/train_imgs/train/'
    # Get all .png files in the directory
    png_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.png')]
    
    # Randomly select 100 .png files
    selected_files = random.sample(png_files, 100)

    # Open all selected images
    images = [Image.open(file) for file in selected_files]
    
    # Determine the size for each small image in the collage (assuming all images are the same size)
    width, height = images[0].size

    # Create a blank 10x10 collage
    collage_width = width * 10
    collage_height = height * 10
    collage = Image.new('RGB', (collage_width, collage_height))

    # Paste each image into the collage
    for index, image in enumerate(images):
        x = (index % 10) * width
        y = (index // 10) * height
        collage.paste(image, (x, y))

    # Save the collage
    collage.save('figure/10-by-10-imgs/'+
                 experiment+'_'+'_'.join(channel_list)+'_'+preprocessing+'.png')
    return

Experiment 01 - Included 5 genes from Rohban et al, reported to be involved in pathways shown to have distinct morphology from DMSO.

In [ ]:
# experiment 1
experiment = 'experiment_01'
gene_list = ['RAC1', 'KRAS', 'CDC42', 'RHOA', 'PAK1']
channel_list1 = ['RNA', 'Mito', 'DNA']

create_experiment_directories(experiment, gene_list, channel_list1)

# create_collage(experiment, channel_list1, 'resized')
# create_collage(experiment, channel_list1, 'cropped')

Experiment - based on Rohban et al 1F supplementary material. Included gene clusters with at least three genes, p-value of first pathway less than 0.01, and at least half of the genes associated with the GO term in the cluster vs. all the clusters. Randomely excluded one gene from each cluster during training to test OOD prediction.

In [ ]:
# experiment 3
experiment = 'experiment_03'
gene_allele_list = ['XBP1', 'MAPK14', # GO:0050663 cytokine secretion
                    'RAC1_T17N', 'AKT1_E17K', 'AKT3', 'AKT3_E17K', # GO:0031294 lymphocyte costimulation
                    'RHOA_Q63L', 'PRKACA', # GO:0007043 cell-cell junction assembly
                    'SMAD4', 'RPS6KB1', # GO:0001657 ureteric bud development
                    'KRAS', 'BRAF', 'RAF1', # GO:0000186 activation of MAPKK activity
                    ]
gene_list = []
for allele in gene_allele_list:
    
    # check if there is any row in df_merged with gene_name or pert_name equal to allele and save its gene_name
    gene_name = df_merged[df_merged["gene_name"] == allele]["gene_name"].values
    allele_name = df_merged[df_merged["pert_name"] == allele]["gene_name"].values
    gene_list_tmp = list(gene_name) + list(allele_name)
    gene_list += list(set(gene_list_tmp))
    print('Allele: '+allele+' - Gene name: ', list(set(gene_list_tmp)))
    
channel_list1 = ['RNA', 'Mito', 'DNA']
print('gene list:', gene_list)

create_experiment_directories(experiment, gene_list, channel_list1)

# create_collage(experiment, channel_list1, 'resized')
# create_collage(experiment, channel_list1, 'cropped')

### create experiment 04 including untreated cells labeled with EMPTY

In [ ]:
# experiment 4
experiment = 'experiment_04'
gene_list = ['EMPTY']
channel_list1 = ['RNA', 'Mito', 'DNA']

create_experiment_directories(experiment, gene_list, channel_list1)

# create_collage(experiment, channel_list1, 'resized')
# create_collage(experiment, channel_list1, 'cropped')

Generate gene embedding from scGPT, from: https://github.com/bowang-lab/scGPT/blob/main/tutorials/Tutorial_GRN.ipynb

In [ ]:
import copy
import json
import os
from pathlib import Path
import sys
import warnings

import torch
from anndata import AnnData
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pandas as pd
import tqdm

from torchtext.vocab import Vocab
from torchtext._torchtext import (
    Vocab as VocabPybind,
)

sys.path.insert(0, "../")
import scgpt as scg
from scgpt.tasks import GeneEmbedding
from scgpt.tokenizer.gene_tokenizer import GeneVocab
from scgpt.model import TransformerModel
from scgpt.preprocess import Preprocessor
from scgpt.utils import set_seed 

os.environ["KMP_WARNINGS"] = "off"
warnings.filterwarnings('ignore')

In [ ]:
set_seed(42)
pad_token = "<pad>"
special_tokens = [pad_token, "<cls>", "<eoc>"]
# n_hvg = 1200
n_bins = 51
mask_value = -1
pad_value = -2
n_input_bins = n_bins

In [ ]:
# Specify model path; here we load the pre-trained scGPT model downloaded from the scGPT repository
model_dir = Path("scgpt/scGPT_human")
model_config_file = model_dir / "args.json"
model_file = model_dir / "best_model.pt"
vocab_file = model_dir / "vocab.json"

vocab = GeneVocab.from_file(vocab_file)
for s in special_tokens:
    if s not in vocab:
        vocab.append_token(s)

# Retrieve model parameters from config files
with open(model_config_file, "r") as f:
    model_configs = json.load(f)
print(
    f"Resume model from {model_file}, the model args will override the "
    f"config {model_config_file}."
)
embsize = model_configs["embsize"]
nhead = model_configs["nheads"]
d_hid = model_configs["d_hid"]
nlayers = model_configs["nlayers"]
n_layers_cls = model_configs["n_layers_cls"]

gene2idx = vocab.get_stoi()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ntokens = len(vocab)  # size of vocabulary
model = TransformerModel(
    ntokens,
    embsize,
    nhead,
    d_hid,
    nlayers,
    vocab=vocab,
    pad_value=pad_value,
    n_input_bins=n_input_bins,
)

try:
    model.load_state_dict(torch.load(model_file, map_location=torch.device('cpu')))
    print(f"Loading all model params from {model_file}")
except:
    # only load params that are in the model and match the size
    model_dict = model.state_dict()
    pretrained_dict = torch.load(model_file, map_location=torch.device('cpu'))
    pretrained_dict = {
        k: v
        for k, v in pretrained_dict.items()
        if k in model_dict and v.shape == model_dict[k].shape
    }
    for k, v in pretrained_dict.items():
        print(f"Loading params {k} with shape {v.shape}")
        model_dict.update(pretrained_dict)
        model.load_state_dict(model_dict)

model.to(device)

In [ ]:
# Retrieve the data-independent gene embeddings from scGPT
gene_ids = np.array([id for id in gene2idx.values()])
gene_embeddings = model.encoder(torch.tensor(gene_ids, dtype=torch.long).to(device))
gene_embeddings = gene_embeddings.detach().cpu().numpy()

In [ ]:
gene_embeddings.shape

In [ ]:
metadata_df['gene_name'].unique()

In [ ]:
gene2idx.keys()

In [ ]:
# Filter on the intersection between the rohban et al genes and scGPT's 30+K foundation model vocab
gene_embeddings_rohban = {gene: gene_embeddings[i] for i, gene in enumerate(gene2idx.keys()) if gene in metadata_df['gene_name'].unique()}
print('Retrieved gene embeddings for {} genes.'.format(len(gene_embeddings_rohban)))
print(type(gene_embeddings_rohban))

In [ ]:
# experiment 01 genes
gene_list = ['RAC1', 'KRAS', 'CDC42', 'RHOA', 'PAK1']

# check if the gene embedding inside gene_list is generated by scGPT
for gene in gene_list:
    if gene in list(gene_embeddings_rohban.keys()):
        continue
    else:
        print(gene)

In [ ]:
import numpy as np
import csv

def write_dict_to_csv(input_dict, filename):
    with open(filename, mode='w', newline='') as file:
        for key, values in input_dict.items():
            
            row = key.lower()
            for value in gene_embeddings_rohban[key]:
                row = row+','+str(value)
            file.write(row+'\n')

write_dict_to_csv(gene_embeddings_rohban, 'required_file/rohban_scgpt_gene_embedding_dict.csv')

In [ ]:
pert_to_embedding = pd.read_csv('required_file/rohban_scgpt_gene_embedding_dict.csv', header=None)

In [ ]:
pert_to_embedding.columns = ['gene_name'] + [str(i) for i in range(512)]

In [ ]:
pert_to_embedding